In [6]:
%load_ext pycodestyle_magic
%pycodestyle_on
#%flake8_on

# Race condition in Python

Incrementing a simple value in one single Thread is easy an produces no errors. 
As soon as you have multiple Threads working with a shared variable, problems can arise. **Race conditions** can occure, this means the value of the shared variable is modified in an erronous way.

Let's have a look at the following example. There is a class `Counter` with a method `increment`. This method does nothing else than incrementing the value by 1.
Multiple threads (here two) share an instance of the object `Counter` and invoke the `increment` method. Both threads do this one million times. Thus the final value of the counter should be 2 millions.

***But after execution you will see that this is not the case*** Why?

In [2]:
from threading import Thread


class Counter(object):
    def __init__(self):
        self.value = 0

    def increment(self):
        self.value += 1


c = Counter()


def go():
    for i in range(1000000):
        c.increment()


# Run two threads that increment the counter:
t1 = Thread(target=go)
t1.start()
t2 = Thread(target=go)
t2.start()
t1.join()
t2.join()
print(c.value)

1698294


## The race condition problem

We incremented 2,000,000 times, but that’s not what we got. The problem is that self.value += 1 actually takes three distinct steps:

1. Getting the attribute,
2. incrementing it,
3. then setting the attribute.

If two threads call `increment()` on the same object around the same time, the following series steps may happen:

```
Thread 1: Get self.value, which happens to be 13.
Thread 2: Get self.value, which happens to be 13.
Thread 1: Increment 13 to 14.
Thread 1: Set self.value to 14.
Thread 2: Increment 13 to 14.
Thread 1: Set self.value to 14.
```
An increment was lost due to a race condition.

We could use LOCKS... but this will be seen later


### Race condition with simple Integer variable
In this example we see that only and increment operation is executed multiple times on `Integer` variable.

If the code is executed with a little number of threads (e.g. 4) no problem happens. The final value is correct. But as soon as we increas the number of *sessions* (or loops) the end value gets buggy. This, because the Python interpreter and also the OS can interrupt the current thread and give the CPU to another thread. Unfortunatelly, this context-switch can happen in the middle of the `x += 1` instruction, which translates into: `LOAD x, INC x, STORE x`

In [9]:
from threading import Thread

S = 4       # number of "sessions"
S = 400000  # number of "sessions"
N = 6       # the number of threads
x = 0       # shared ressource


def my_process():
    for i in range(S):
        global x
        # print("Session ", i)

        # non-critical section
        # ...
        # entry protocol
        # critical section
        x += 1
        # exit protocol


threads = []
for j in range(N):
    thread = Thread(target=my_process)
    threads.append(thread)
    thread.start()

for thread in threads:  # iterates over the threads
    thread.join()       # waits until the thread has finished work

print(x)

837799


### Critical section protected by a LOCK


In [3]:
from threading import Thread
from threading import Lock


class Counter(object):
    def __init__(self):
        self.value = 0
        self.lock = Lock()

    def increment(self):
        with self.lock:
            self.value += 1


c = Counter()


def go():
    for i in range(1000000):
        c.increment()


# Run two threads that increment the counter:
t1 = Thread(target=go)
t1.start()
t2 = Thread(target=go)
t2.start()
t1.join()
t2.join()
print(c.value)

2000000
